# Macrophage adhesion anlysis
by Felix Romer

last changed: 26.4.2025

## Package import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import statsmodels.formula.api as smf

# Load and Prepare Data
This section loads the dataset and prepares it for analysis by defining base colors and calculating relative metrics.

In [ ]:
load_path = '/path/to/main/directory'  # Update this to your actual path
df = pd.read_csv(f'{load_path}/AdhesionData.csv') 

# Base colors for conditions
base_colors = {
    'Control': '#7c8083', 
    'Infected': '#c53c37',  
    'Uninfected': '#518fba' 
}

# Calculate Relative Metrics
This section calculates the mean and relative macrophage (MF) counts for further analysis.

In [ ]:
# Calculate the mean MF count for uninfected samples for each Chip group
mean_uninfected = df[df['Infection Status'] == 'Uninfected'].groupby('Chip')['MF count'].mean()

# Calculate the relative MF count
df['Relative MF count'] = df.apply(
    lambda row: row['MF count'] / mean_uninfected[row['Chip']] if row['Chip'] in mean_uninfected else None, axis=1
)

df_FR = df[df['Chip'] != 'Marie']
# grop for chip and channel and get the mean
# Group by Chip and Channel and get the mean for both Relative MF count and MF count
df_FR_mean = df_FR.groupby(['Chip', 'Channel', 'Infection Status']).agg({'Relative MF count': 'mean', 'MF count': 'mean'}).reset_index()
df_FR_mean

## Visalize realtive adhesion
This section saves the generated visualizations as SVG files for further use.

In [ ]:
# Plot relative MF count without hue for boxplot
fig = plt.figure(figsize=(6, 8))
sns.boxplot(x='Infection Status', y='Relative MF count', data=df_FR_mean, palette=base_colors,  width=0.3, whis=[0, 100])
sns.swarmplot(x='Infection Status', y='Relative MF count', data=df_FR_mean, size=5, hue='Chip', dodge=False, palette=gray_colors, edgecolor='black', linewidth=1)

# save as svg
plt.savefig(f'{load_path}/Relative_MF_count.svg', format='svg')


# Statistical Analysis: Relative MF Count
This section performs t-tests and fits a mixed effects model for relative MF counts.

In [16]:
from scipy.stats import ttest_ind

# Separate the data into infected and uninfected groups
infected = df_FR_mean[df_FR_mean['Infection Status'] == 'Infected']['Relative MF count']
uninfected = df_FR_mean[df_FR_mean['Infection Status'] == 'Uninfected']['Relative MF count']

# Perform the t-test
t_stat, p_value = ttest_ind(infected, uninfected)
print('t-statistic:', t_stat)
print('p-value:', p_value)

# Determine if the null hypothesis can be rejected
if p_value < 0.05:
    print('Reject the null hypothesis')
else:
    print('Fail to reject the null hypothesis')
# Perform the t-test for each chip
for chip in df_FR_mean['Chip'].unique():
    infected_chip = df_FR_mean[(df_FR_mean['Infection Status'] == 'Infected') & (df_FR_mean['Chip'] == chip)]['Relative MF count']
    uninfected_chip = df_FR_mean[(df_FR_mean['Infection Status'] == 'Uninfected') & (df_FR_mean['Chip'] == chip)]['Relative MF count']
    
    if len(infected_chip) > 0 and len(uninfected_chip) > 0:
        t_stat_chip, p_value_chip = ttest_ind(infected_chip, uninfected_chip)
        print('---------------------------------')
        print(f'Chip: {chip}')
        print('t-statistic:', t_stat_chip)
        print('p-value:', p_value_chip)
        
        if p_value_chip < 0.05:
            print('Reject the null hypothesis')
        else:
            print('Fail to reject the null hypothesis')


t-statistic: 3.7129467904992777
p-value: 0.002604960319757532
Reject the null hypothesis
---------------------------------
Chip: 16
t-statistic: 3.4780417182012626
p-value: 0.040112054463393276
Reject the null hypothesis
---------------------------------
Chip: 19
t-statistic: 3.436969682882612
p-value: 0.00886329980139118
Reject the null hypothesis


In [10]:
# rename columns ' ' to '_'
df_FR_mean.columns = [col.replace(' ', '_') for col in df_FR_mean.columns]
# Mixed Effects Model: MF_count ~ Infection_Status + (1|Chip)
model = smf.mixedlm("Relative_MF_count ~ Infection_Status", data=df_FR_mean, groups=df_FR_mean["Chip"], re_formula="1")
result = model.fit()

# Reverse column names to original
df_FR_mean.columns = [col.replace('_', ' ') for col in df_FR_mean.columns]
# Ergebnisse ausgeben
print(result.summary())
# print p value
print(result.pvalues)

                  Mixed Linear Model Regression Results
Model:                MixedLM    Dependent Variable:    Relative_MF_count
No. Observations:     15         Method:                REML             
No. Groups:           2          Scale:                 0.0613           
Min. group size:      5          Log-Likelihood:        -2.5829          
Max. group size:      10         Converged:             Yes              
Mean group size:      7.5                                                
-------------------------------------------------------------------------
                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                       1.476    0.107 13.803 0.000  1.267  1.686
Infection_Status[T.Uninfected] -0.495    0.129 -3.835 0.000 -0.748 -0.242
Group Var                       0.007    0.094                           

Intercept                         2.445856e-43
Infectio

/home/felix/miniconda3/envs/statenv/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
